In [113]:
import torch
import timm
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as datasets
from torchvision import transforms as ts
import torch.nn.functional as F
import torch.nn as nn
from torchvision.transforms import ToTensor
from tqdm import tqdm
import matplotlib.pyplot as plt

In [114]:
# Loading the dataset 
train_path ='D:/Data/Datasets/30vnf_Food_Images/Train'
valid_path = 'D:/Data/Datasets/30vnf_Food_Images/Validate'
test_path = 'D:/Data/Datasets/30vnf_Food_Images/Test'


In [115]:
trf = ts.Compose([ts.Resize((128, 128)),
                ts.ToTensor(),
                ts.RandomRotation(degrees=15),
                ts.RandomHorizontalFlip(),
                ts.Normalize(0.5, 0.5)])

In [116]:
def load_data(train, test, valid):
    
    train = datasets.ImageFolder(root=train_path, transform=trf)
    test = datasets.ImageFolder(root=test_path, transform=trf)
    valid = datasets.ImageFolder(root=valid_path, transform=trf)

    print(f'Train dataset: {len(train)}')
    print(f'Validation dataset: {len(valid)}')
    print(f'Test dataset: {len(test)}')
    print(f'Number of classes: {len(train.classes)}')

    return train, test, valid  

train_dataset, val_dataset, test_dataset = load_data(train_path,test_path,valid_path)


Train dataset: 17581
Validation dataset: 2515
Test dataset: 5040
Number of classes: 30


In [117]:
print(train_dataset[0][0].shape)

torch.Size([3, 128, 128])


In [118]:
# Loading the dataloader
bs=64
train_dl = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=2)
val_dl = DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=2)
test_dl = DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=2)

print(len(train_dl))
print(len(val_dl))
print(len(test_dl))    


275
79
40


In [119]:
im_size = 128
in_chs = 3
out_chs = 16
ks = 3
pad = 1 
pool_size = 2
num_classes = 30

in_fs = (im_size // 32) * (im_size // 32)* out_chs
print(in_fs)

256


In [120]:
class mynet(nn.Module):
    def __init__(self, in_chs, out_chs, in_fs, ks, pad, ps, n_cls):
        super(mynet, self).__init__()
        
        self.in_chs = in_chs,
        self.out_chs = out_chs,
        self.in_fs = in_fs,
        self.ks = ks,
        self.pad = pad,
        self.ps = ps,
        self.n_cls = n_cls
        
        self.conv1 = nn.Conv2d(in_chs,out_chs,kernel_size=ks,stride=ps,padding=pad)
        self.conv2 = nn.Conv2d(out_chs,out_chs*2,kernel_size=ks,stride=ps,padding=pad)
        self.conv3 = nn.Conv2d(out_chs*2,out_chs*4,kernel_size=ks,stride=ps,padding=pad)
        self.conv4 = nn.Conv2d(out_chs*4,out_chs*4,kernel_size=ks,stride=ps,padding=pad)
        self.pool = nn.MaxPool2d(kernel_size = ps, stride = ps)
        self.dropout = nn.Dropout2d(0.4)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.batchnorm3 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(in_features=in_fs, out_features=in_fs//2 )
        self.fc2 = nn.Linear(in_features=in_fs//2, out_features=in_fs//4)
        self.fc3 = nn.Linear(in_features=in_fs//4, out_features=n_cls)
        self.softmax = torch.nn.Softmax(dim = 1)
        
        
    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.dropout(self.batchnorm2(self.pool(x)))
        x = self.batchnorm3(self.pool(F.relu(self.conv3(x))))
        x = self.dropout(self.conv4(x))
        bs= x.shape[0]
        x = x.view(bs, -1) # Flatten layer
        x = self.dropout(self.fc1(x))
        x = self.dropout(self.fc2(x))
        x = self.softmax(self.fc3(x))
        
        return x
    
m = mynet(in_chs = in_chs, out_chs = out_chs, in_fs = in_fs, ks = ks, pad = pad, ps = pool_size, n_cls = num_classes)
m

mynet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout2d(p=0.4, inplace=False)
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=30, bias=True)
  (softmax): Softmax(dim=1)
)

In [121]:
y = torch.rand(1, in_chs, im_size, im_size)
m(y)

tensor([[0.0311, 0.0286, 0.0404, 0.0350, 0.0366, 0.0379, 0.0363, 0.0398, 0.0425,
         0.0306, 0.0289, 0.0312, 0.0410, 0.0404, 0.0245, 0.0302, 0.0194, 0.0241,
         0.0336, 0.0277, 0.0384, 0.0287, 0.0414, 0.0232, 0.0256, 0.0342, 0.0413,
         0.0441, 0.0311, 0.0323]], grad_fn=<SoftmaxBackward0>)

In [122]:
def train_setup(m): return m.to("cuda"), 20, "cuda", torch.nn.CrossEntropyLoss(), torch.optim.Adam(params = m.parameters(), lr = 0.001)

m, epochs, device, loss_fn, optimizer = train_setup(m)

In [123]:
torch.cuda.empty_cache()

In [124]:
patience = 5  
best_val_loss = float('inf')
best_val_acc = 0
improvement = 0

train_losses, train_accs = [], []
val_losses, val_accs = [], []


for epoch in range(epochs):
    
    epoch_loss, epoch_acc, total = 0, 0, 0
    for idx, batch in tqdm(enumerate(train_dl)):
        ims, gts = batch
        ims, gts = ims.to(device), gts.to(device)
        
        total += ims.shape[0]
        
        preds = m(ims)
        loss = loss_fn(preds, gts)
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, pred_cls = torch.max(preds.data, dim = 1)
        epoch_acc += (pred_cls == gts).sum().item()
        epoch_loss += loss.item()
        
    
    tr_loss = epoch_loss / len(train_dl)
    train_losses.append(tr_loss)
    train_accs.append(epoch_acc/total)
    
    print(f"Epoch {epoch + 1} train process is finished")
    print(f"Epoch {epoch + 1} train loss -> {(tr_loss):.3f}")
    print(f"Epoch {epoch + 1} train accuracy -> {(epoch_acc / total):.3f}")
    
    m.eval()
    with torch.no_grad():
        val_epoch_loss, val_epoch_acc, val_total = 0, 0, 0
        for idx, batch in enumerate(val_dl):
            ims, gts = batch
            ims, gts = ims.to(device), gts.to(device)
            val_total += ims.shape[0]

            
            preds = m(ims)
            loss = loss_fn(preds, gts)

            _, pred_cls = torch.max(preds.data, dim = 1)
            val_epoch_acc += (pred_cls == gts).sum().item()
            val_epoch_loss += loss.item()
        
        val_acc = val_epoch_acc / val_total
        val_loss = val_epoch_loss / len(val_dl)
        
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print(f"Epoch {epoch + 1} validation is finished")
        print(f"Epoch {epoch + 1} validation loss -> {(val_loss):.3f}")
        print(f"Epoch {epoch + 1}  validation accuracy -> {val_acc:.3f}")
        
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            improvement = 0
            torch.save(m.state_dict(), 'Custom_model.pth')
        else:
            improvement += 1
            if improvement >= patience:
                print(f"No improvement in validation loss for {patience} epochs. Stopping early.")
                break


275it [03:04,  1.49it/s]


Epoch 1 train process is finished
Epoch 1 train loss -> 3.353
Epoch 1 train accuracy -> 0.098
Epoch 1 validation is finished
Epoch 1 validation loss -> 3.344
Epoch 1  validation accuracy -> 0.110


275it [04:25,  1.04it/s]


Epoch 2 train process is finished
Epoch 2 train loss -> 3.327
Epoch 2 train accuracy -> 0.128
Epoch 2 validation is finished
Epoch 2 validation loss -> 3.340
Epoch 2  validation accuracy -> 0.115


275it [04:21,  1.05it/s]


...

Epoch 6 train process is finished
Epoch 6 train loss -> 3.396
Epoch 6 train accuracy -> 0.061
Epoch 6 validation is finished
Epoch 6 validation loss -> 3.396
Epoch 6  validation accuracy -> 0.061


275it [04:37,  1.01s/it]


Epoch 7 train process is finished
Epoch 7 train loss -> 3.395
Epoch 7 train accuracy -> 0.062
Epoch 7 validation is finished
Epoch 7 validation loss -> 3.395
Epoch 7  validation accuracy -> 0.062


275it [03:48,  1.20it/s]


Epoch 8 train process is finished
Epoch 8 train loss -> 3.396
Epoch 8 train accuracy -> 0.061
Epoch 8 validation is finished
Epoch 8 validation loss -> 3.396
Epoch 8  validation accuracy -> 0.061
No improvement in validation loss for 5 epochs. Stopping early.


# Training on kernel_size of 3

In [125]:
275it [04:41,  1.02s/it]
Epoch 1 train process is finished
Epoch 1 train loss -> 3.364
Epoch 1 train accuracy -> 0.086
Epoch 1 validation is finished
Epoch 1 validation loss -> 3.355
Epoch 1  validation accuracy -> 0.093
275it [02:37,  1.75it/s]
Epoch 2 train process is finished
Epoch 2 train loss -> 3.333
Epoch 2 train accuracy -> 0.119
Epoch 2 validation is finished
Epoch 2 validation loss -> 3.328
Epoch 2  validation accuracy -> 0.123
275it [02:35,  1.77it/s]
Epoch 3 train process is finished
Epoch 3 train loss -> 3.315
Epoch 3 train accuracy -> 0.138
Epoch 3 validation is finished
Epoch 3 validation loss -> 3.312
Epoch 3  validation accuracy -> 0.141
275it [02:37,  1.75it/s]
Epoch 4 train process is finished
Epoch 4 train loss -> 3.297
Epoch 4 train accuracy -> 0.157
Epoch 4 validation is finished
Epoch 4 validation loss -> 3.304
Epoch 4  validation accuracy -> 0.150
275it [02:35,  1.77it/s]
Epoch 5 train process is finished
Epoch 5 train loss -> 3.277
Epoch 5 train accuracy -> 0.178
Epoch 5 validation is finished
Epoch 5 validation loss -> 3.274
Epoch 5  validation accuracy -> 0.182
275it [02:33,  1.79it/s]
Epoch 6 train process is finished
Epoch 6 train loss -> 3.268
Epoch 6 train accuracy -> 0.188
Epoch 6 validation is finished
Epoch 6 validation loss -> 3.271
Epoch 6  validation accuracy -> 0.183
275it [29:16,  6.39s/it] 
Epoch 7 train process is finished
Epoch 7 train loss -> 3.265
Epoch 7 train accuracy -> 0.190
Epoch 7 validation is finished
Epoch 7 validation loss -> 3.265
Epoch 7  validation accuracy -> 0.190
275it [02:33,  1.79it/s]
Epoch 8 train process is finished
Epoch 8 train loss -> 3.256
Epoch 8 train accuracy -> 0.199
Epoch 8 validation is finished
Epoch 8 validation loss -> 3.263
Epoch 8  validation accuracy -> 0.194
275it [02:36,  1.76it/s]
Epoch 9 train process is finished
Epoch 9 train loss -> 3.249
Epoch 9 train accuracy -> 0.205
Epoch 9 validation is finished
Epoch 9 validation loss -> 3.265
Epoch 9  validation accuracy -> 0.190
275it [04:20,  1.06it/s]
Epoch 10 train process is finished
Epoch 10 train loss -> 3.249
Epoch 10 train accuracy -> 0.206
Epoch 10 validation is finished
Epoch 10 validation loss -> 3.268
Epoch 10  validation accuracy -> 0.187
275it [03:53,  1.18it/s]
Epoch 11 train process is finished
Epoch 11 train loss -> 3.248
Epoch 11 train accuracy -> 0.207
Epoch 11 validation is finished
Epoch 11 validation loss -> 3.265
Epoch 11  validation accuracy -> 0.192
275it [03:31,  1.30it/s]
Epoch 12 train process is finished
Epoch 12 train loss -> 3.248
Epoch 12 train accuracy -> 0.207
Epoch 12 validation is finished
Epoch 12 validation loss -> 3.265
Epoch 12  validation accuracy -> 0.191
No improvement in validation loss for 4 epochs. Stopping early.
Train dataset: 17581
Valid

SyntaxError: invalid decimal literal (3136133978.py, line 1)

# Training on kernel_size of 5

In [ ]:
275it [03:54,  1.17it/s]
Epoch 1 train process is finished
Epoch 1 train loss -> 3.361
Epoch 1 train accuracy -> 0.094
Epoch 1 validation is finished
Epoch 1 validation loss -> 3.343
Epoch 1  validation accuracy -> 0.112
275it [03:58,  1.15it/s]
Epoch 2 train process is finished
Epoch 2 train loss -> 3.357
Epoch 2 train accuracy -> 0.100
Epoch 2 validation is finished
Epoch 2 validation loss -> 3.360
Epoch 2  validation accuracy -> 0.097
275it [03:05,  1.49it/s]
Epoch 3 train process is finished
Epoch 3 train loss -> 3.347
Epoch 3 train accuracy -> 0.109
Epoch 3 validation is finished
Epoch 3 validation loss -> 3.380
Epoch 3  validation accuracy -> 0.076
275it [05:12,  1.14s/it]
Epoch 4 train process is finished
Epoch 4 train loss -> 3.350
Epoch 4 train accuracy -> 0.106
Epoch 4 validation is finished
Epoch 4 validation loss -> 3.351
Epoch 4  validation accuracy -> 0.105
275it [05:10,  1.13s/it]
Epoch 5 train process is finished
Epoch 5 train loss -> 3.347
Epoch 5 train accuracy -> 0.108
Epoch 5 validation is finished
Epoch 5 validation loss -> 3.349
Epoch 5  validation accuracy -> 0.107
275it [03:18,  1.39it/s]
Epoch 6 train process is finished
Epoch 6 train loss -> 3.342
Epoch 6 train accuracy -> 0.114
Epoch 6 validation is finished
Epoch 6 validation loss -> 3.358
Epoch 6  validation accuracy -> 0.099
No improvement in validation loss for 5 epochs. Stopping early.
  Cell In[45], line 1
    275it [04:41,  1.02s/it]
      ^
SyntaxError: invalid decimal literal